# Erklärung Transformation

Dieser Abschnitt erklärt die meisten Methoden des Objektes Transformation. Weggelassen wurden Methoden, welche nur für die Entwicklung benötigt wurden und im Produktivcode nicht aufgerufen werden (Debugging Methoden).
### Disclaimer
Der Code ist nicht lauffähig, dafür existieren die Python Skripte. Das Notebook stellt lediglich eine Dokumentation dar.

### Verbesserungen, welche an SlidingWindows vorgenommen wurden
- Die Position des Rechteckes für die Transformation wurde selbst optimiert

### Import und Klassendeklaration

Hier finden die spezifischen Imports statt. Wichtig sind hier die Calibration als plt.

In [ ]:
import time
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np

from calib import Calibration as cal

class Transformation():

    def __init__(self, debug = False) -> None:
        self.debug = debug

### calculate matrix
In dieser Methode wird die Transformationsmatrix und ihre Inverse berechnet. Diese wird benötigt, um das Bild in Vogelperspektive zu transformieren, oder ein verarbeitetes Bild mit der Inversen wieder in die 'normale' Perspektive zurück zu transformieren. Wichtig ist die Abgrenzung zur Bild-Entzerrung, diese wird in Calib vorgenommen. 

In [ ]:
def _calculate_matrix(self, img, trans_matrix):
    src_coor = trans_matrix['SRC']
    dst_coor = trans_matrix['DST']
    src = np.float32([src_coor[0], src_coor[1], src_coor[2], src_coor[3]]) 
    dst = np.float32([dst_coor[0], dst_coor[1], dst_coor[2], dst_coor[3]])

    M = cv.getPerspectiveTransform(src,dst)
    M_reversed = cv.getPerspectiveTransform(dst,src)
    return M, M_reversed

### transform image perspective
Diese Methode transformiert ein eingehendes Bild mit der Matrix aus calculate_matrix() in die Vogelperspektive. Um die Rücktransformation möglich zu machen, wird die Inverse der Matrix beim Aufruf ebenfalls zurückgegeben. Die gewählten Punkte für den Transformationsbereich wurden selbst herausgesucht (Mit Orientierung an den Positionen welche in der Projektbeschreibung vorgegeben waren). Die Koordinaten für die Matrixerstellung sind in der Konfigurationsdatei gespeichert, um ein simples Anpassen zu ermöglichen. Hier 3 Beispiele, wo die Punkte auf der Spur positioniert wurden:

<img src="Images/perspective_transformation_example_1.png">
<img src="Images/perspective_transformation_example_2.png">
<img src="Images/perspective_transformation_example_3.png">

In [ ]:
def transform_image_perspective(self, img, trans_matrix):
    M, M_reversed = self._calculate_matrix(img, trans_matrix)

    img_transformed = cv.warpPerspective(img,M,(img.shape[1], img.shape[0]),flags=cv.INTER_LINEAR) # Alternative flag: INTER_NEAREST

    return img_transformed, M_reversed